In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2, Xception, ResNet50, ResNet101, InceptionV3, VGG16,ResNet50V2
from keras.preprocessing.image import load_img, img_to_array
import os
import sklearn
from sklearn.metrics import classification_report
import tensorflow_addons as tfa
import seaborn as sns
from collections import Counter
import operator

In [3]:
IMG_SIZE = 240
IMG_SIZE_2 = 330
TARGET_SIZE = (IMG_SIZE, IMG_SIZE_2)
IMG_SHAPE = (IMG_SIZE, IMG_SIZE_2, 3)
NUM_CLASSES = 40

datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    featurewise_center=True,
    horizontal_flip = False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)
batch_size = 30
# load and iterate training dataset
train_it = datagen.flow_from_directory('C:/Users/avuat/Python Projects/Motorcycle Classifier/images/TrainValidationWithSplitModelRE/train/', class_mode='categorical', batch_size=batch_size, target_size=TARGET_SIZE)
val_it = datagen.flow_from_directory('C:/Users/avuat/Python Projects/Motorcycle Classifier/images/TrainValidationWithSplitModelRE/validation/', class_mode='categorical', batch_size=batch_size, target_size=TARGET_SIZE)
print(Counter(train_it.labels))

Found 5755 images belonging to 40 classes.
Found 469 images belonging to 40 classes.
Counter({17: 371, 16: 317, 3: 313, 24: 298, 6: 286, 14: 268, 35: 254, 23: 253, 27: 250, 9: 227, 30: 226, 22: 177, 34: 174, 37: 169, 12: 159, 5: 151, 18: 148, 32: 135, 11: 133, 1: 131, 10: 98, 33: 95, 4: 94, 36: 92, 39: 87, 0: 86, 31: 83, 38: 82, 25: 71, 20: 64, 28: 64, 19: 61, 8: 49, 13: 47, 7: 46, 15: 46, 29: 41, 21: 40, 26: 35, 2: 34})


In [4]:
inputs = tf.keras.Input(shape = IMG_SHAPE)
base_model = ResNet50V2(weights='imagenet', include_top=True, input_tensor=inputs)

last_layer = base_model.layers[-2].output 
out = tf.keras.layers.Dense(units = NUM_CLASSES, activation = 'softmax', name = 'ouput')(last_layer)
new_base_model = tf.keras.Model(inputs = inputs, outputs = out)

for layer in new_base_model.layers[:-20]:
    layer.trainable = False

new_base_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="categorical_crossentropy",
              metrics=['accuracy'])
              
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',mode = 'min')       

In [ ]:
history = new_base_model.fit(train_it, steps_per_epoch=int(5755/batch_size),validation_data=val_it, validation_steps=int(469/batch_size), epochs=20)

C:\Users\avuat\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/20
142/191 [=====================>........] - ETA: 2:21 - loss: 1.5891 - accuracy: 0.5351

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
new_base_model.compile(metrics=[
                       "accuracy",
                       tf.keras.metrics.AUC(num_thresholds=10000),
                       tf.keras.metrics.Precision(thresholds=0.5),
                       tf.keras.metrics.Recall(thresholds=0.5),
                       tfa.metrics.F1Score(num_classes=NUM_CLASSES, threshold=0.5)
                       ])

results = new_base_model.evaluate(val_it,return_dict=True)

In [ ]:
for i, value in enumerate(results["f1_score"]):
    print(list(val_it.class_indices)[i], " : ", round(value*100, 2), "%")

In [ ]:
y_val = []
y_pred = []
for i in range(0,int(469/batch_size)):
    y_val += val_it[i][1].argmax(axis=1).tolist()
    y_pred += new_base_model.predict(val_it[i][0].reshape(batch_size, TARGET_SIZE[0], TARGET_SIZE[1], 3)).argmax(axis=1).tolist()

print(len(y_val))
print(len(y_pred))

In [ ]:
print("Accuracy : ",round(sum(map(operator.eq, y_pred, y_val)) / len(y_pred)*100, 4), "%")
matrix = sklearn.metrics.confusion_matrix(y_pred, y_val)
plt.subplots(figsize=(20,15))
sns.heatmap(matrix, annot=True,xticklabels=train_it.class_indices, yticklabels=train_it.class_indices)

In [ ]:
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=TARGET_SIZE)
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, TARGET_SIZE[0], TARGET_SIZE[1], 3)
    # prepare pixel data
    img = img.astype('float32')
    img = img / 255.0
    return img 



In [ ]:
directory = 'C:/Users/avuat/Python Projects/Motorcycle Classifier/images/TrainValidationWithSplitModelRE/test/'
f, axarr = plt.subplots(10,2,figsize=(50,60))
f.tight_layout()
i = 0
for filename in os.listdir(directory):
    if int(i/2) >= 20 :
        break
    img= load_image(os.path.join(directory, filename) )
    result = new_base_model.predict(img)
    #print("Predicted : "+list(train_it.class_indices)[result.argmax(axis=-1)[0]])
    axarr[int(i/4), i%4].imshow(img[0])
    axarr[int(i/4), i%4].title.set_text(list(train_it.class_indices)[result.argmax(axis=-1)[0]])
    i += 1
    #axarr[0,0].show()
    


In [ ]:
train_it.class_indices